In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

os.chdir('../data/TewheyMPRA')

from pyfaidx import Fasta
from collections import Counter

In [6]:
# cell_type = 'GM12878'
# cell_type = 'A549'
cell_type = 'HCT116'

In [7]:
df_list = []
for file_name in os.listdir(cell_type):
    if file_name.endswith('.tsv'):
        df = pd.read_csv(f'{cell_type}/{file_name}', sep='\t', low_memory=False)

        c = Counter(df['ctrl_exp'])
        print(c.most_common())
        df['experiment'] = c.most_common()[0][0]
        ctrl_pos_mean = df[df['ctrl_exp'] == 'ctrl_pos']['log2FoldChange'].mean()
        ctrl_neg_mean = df[df['ctrl_exp'] == 'ctrl_neg']['log2FoldChange'].mean()
        total_mean = df['log2FoldChange'].mean()
        print(ctrl_pos_mean, ctrl_neg_mean, total_mean)
        df['log2FoldChangeOverControl'] = (df['log2FoldChange'] - ctrl_neg_mean) / (ctrl_pos_mean - ctrl_neg_mean)
        df_list.append(df)
df = pd.concat(df_list).reset_index(drop=True)
df

# 不能简单合并，要按照ctrl做归一化再合并
# 这里把ctrl_pos归一化到1，把ctrl_neg归一化到0，计算log2FoldChangeOverControl



df['chr'] = df['chr'].apply(lambda x: 'chr' + x)
df.rename(columns={
    'ref_allele': 'ref', 
    'alt_allele': 'alt', 
    }, inplace=True)

df = df.sort_values(by=['chr', 'pos']).reset_index(drop=True)

# 只保留ID只有5个:的
df = df[df['ID'].str.count(':') == 5].reset_index(drop=True)
# 只保留window是center的
df = df[df['window'] == 'center'].reset_index(drop=True)
# 只保留allele长度为1的或者allele为nan的
df = df[((df['ref'].str.len() == 1) & (df['alt'].str.len() == 1)) | (df['ref'].isna())].reset_index(drop=True)
df




# 合并相同ID的序列，活性使用平均值
# df = df.groupby(['ID'], as_index=False).agg(lambda x: x.mean() if x.dtype == float else x.iloc[0]).reset_index(drop=True)
df = df.groupby('ID', as_index=False).agg(
    {col: 'mean' if df[col].dtype == 'float' else 'first' for col in df.columns}
)
# 删除重复'chr', 'pos', 'allele'的序列，注意这些序列的ID不同，所以是同一个位点，做了两次突变，生成了4条序列的情况，直接删除后两条
df = df.drop_duplicates(subset=['chr', 'pos', 'allele'])
df = df.sort_values(by=['chr', 'pos', 'allele']).reset_index(drop=True)
df





genome = Fasta('/home/hxcai/genome/hg19.fa')

def process_row(row):
    chr, pos, ref, alt, allele = row['chr'], row['pos'], row['ref'], row['alt'], row['allele']

    genome_ref_base = genome[chr][pos-1].seq.upper()
    correct_flag = genome_ref_base == ref  # 位置正确的标志

    seq = genome[chr][pos-100:pos+100].seq.upper()
    if allele != ref:
        seq = seq[:99] + alt + seq[100:]

    return seq, correct_flag

df[['seq', 'flags']] = df.apply(process_row, axis=1, result_type='expand')
print(len(df), df['flags'].sum())



df = df[['ID', 'chr', 'pos', 'allele', 'seq', 'log2FoldChange', 'log2FoldChangeOverControl']]
df.to_csv(f'TewheyMPRA_{cell_type}.csv', index=False)

[('OL41B', 242177), ('ctrl_neg', 506), ('ctrl_pos', 91)]
3.0855209378503883 0.17389626743795658 0.3594951115071403
[('OL42', 251221), ('ctrl_neg', 506), ('ctrl_pos', 91)]
3.016368974717195 0.15183169335106786 0.5271926355812913
406808 406808
